In [1]:
import json
import os
import pandas as pd

In [2]:
data_dir="C:/Users/TEMP/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/"

In [3]:
os.chdir(data_dir)

In [50]:
df= pd.read_csv('Baby Toolbox_Data_Export.tsv', sep='\t') 

C:\Users\TEMP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
df.shape

(107784, 106)

In [60]:
df.isnull()

2155680

In [51]:
keep = [col for col in df.columns if 'AOI' in col]
print(keep)

['AOI[Calib Top Left]Hit', 'AOI[WholeScreen]Hit', 'AOI[Calib Bottom Right]Hit', 'AOI[WholeScreen]Hit.1', 'AOI[Calib Bottom Left]Hit', 'AOI[WholeScreen]Hit.2', 'AOI[Calib Top Right]Hit', 'AOI[WholeScreen]Hit.3', 'AOI[LeftStim_Shoulder]Hit', 'AOI[RightStim_Hose]Hit', 'AOI[WholeScreen]Hit.4', 'AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5', 'AOI[LeftStim_Rooster]Hit', 'AOI[RightStim_Playground]Hit', 'AOI[WholeScreen]Hit.6', 'AOI[LeftStim_Bike]Hit', 'AOI[RightStim_Sneaker]Hit', 'AOI[WholeScreen]Hit.7']


In [39]:
df.shape

(107784, 20)

In [52]:
df=df[keep]

In [67]:
df.describe()

,AOI[Calib Top Left]Hit,AOI[WholeScreen]Hit,AOI[Calib Bottom Right]Hit,AOI[WholeScreen]Hit.1,AOI[Calib Bottom Left]Hit,AOI[WholeScreen]Hit.2,AOI[Calib Top Right]Hit,AOI[WholeScreen]Hit.3,AOI[LeftStim_Shoulder]Hit,AOI[RightStim_Hose]Hit,AOI[WholeScreen]Hit.4,AOI[LeftStim_Ladder]Hit,AOI[RightStim_Melon]Hit,AOI[WholeScreen]Hit.5,AOI[LeftStim_Rooster]Hit,AOI[RightStim_Playground]Hit,AOI[WholeScreen]Hit.6,AOI[LeftStim_Bike]Hit,AOI[RightStim_Sneaker]Hit,AOI[WholeScreen]Hit.7
count,240.000000,240.0,320.000000,320.0,320.000000,320.0,240.000000,240.0,1042.000000,1042.000000,1042.000000,960.000000,960.000000,960.000000,961.000000,961.000000,961.000000,1041.000000,1041.000000,1041.000000
mean,0.845833,1.0,0.684375,1.0,0.650000,1.0,0.941667,1.0,0.519194,0.327255,0.937620,0.183333,0.792708,0.976042,0.517170,0.326743,0.864724,0.761768,0.123919,0.931796
std,0.361863,0.0,0.465492,0.0,0.477717,0.0,0.234862,0.0,0.499871,0.469437,0.241961,0.387141,0.405578,0.152999,0.499965,0.469267,0.342196,0.426207,0.329648,0.252216
min,0.000000,1.0,0.000000,1.0,0.000000,1.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.0,0.000000,1.0,0.000000,1.0,1.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
50%,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# json file

In [38]:
data_dir="C:/Users/TEMP/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"

In [39]:
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [40]:
cols=['eventName','gazeEngineState','gazeLocationName', 'gazeLocationOnScreen', 'gazeLocationPercentX',
 'gazeLocationPercentY', 'lookAtPointX', 'lookAtPointY', 'gazeLocationZoneSet']

In [41]:
df=df[cols]

In [42]:
df.eventName.unique()

array(['beganCalibration', 'data', 'beganCalibrationRound1',
       'faceVerticesChanged', 'presentedCalibrationItem',
       'cameraImageCaptured', 'lookDetected', 'completedCalibrationItem',
       'calibrationAlignmentCheckPassed', 'completedCalibration',
       'beganLiveTest', 'presentedLiveItem', 'gazeLocationChanged',
       'audioStarted', 'audioCompleted', 'beganLookDetection',
       'endedLookDetection', 'completedLiveItem', 'presentedFillerScreen',
       'removedFillerScreen', 'faceLeftScene', 'faceReturnedToScene',
       'completedLiveTest', 'beganDriftPresentationRound',
       'presentedDriftCalibrationItem', 'completedDriftCalibrationItem',
       'completedDriftPresentationRound', 'instrumentCompleted'],
      dtype=object)

In [43]:
df.gazeLocationOnScreen.unique()

array([nan, 'false', 'true'], dtype=object)

In [44]:
mni =df[(df.gazeLocationOnScreen.notnull()) & (df.gazeEngineState=='testing')]
mni

,eventName,gazeEngineState,gazeLocationName,gazeLocationOnScreen,gazeLocationPercentX,gazeLocationPercentY,lookAtPointX,lookAtPointY,gazeLocationZoneSet
602,faceVerticesChanged,testing,left,true,0.393750,-0.417391,61.645709,1.100488,NaN
603,faceVerticesChanged,testing,left,true,0.392083,-0.420264,61.498690,1.115890,NaN
606,faceVerticesChanged,testing,left,true,0.390653,-0.419409,61.356951,1.107581,NaN
607,faceVerticesChanged,testing,left,true,0.389273,-0.420439,58.411147,1.416698,NaN
608,faceVerticesChanged,testing,left,true,0.360605,-0.403290,47.173716,1.709210,NaN
...,...,...,...,...,...,...,...,...,...
19748,faceVerticesChanged,testing,away,true,0.552984,1.419435,77.386349,23.424244,NaN
19749,faceVerticesChanged,testing,away,true,0.545271,1.426689,76.830372,23.525752,NaN
19750,faceVerticesChanged,testing,away,true,0.539860,1.432821,76.626584,23.223443,NaN
19751,faceVerticesChanged,testing,away,true,0.537877,1.407613,77.099562,22.770658,NaN


In [45]:
#Finding the index for presentLiveItem and completeLiveItem to subset the pieces we need for analysis
begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)

In [46]:
end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

In [47]:
df[df['eventName'].str.contains('completedLiveItem')]

,eventName,gazeEngineState,gazeLocationName,gazeLocationOnScreen,gazeLocationPercentX,gazeLocationPercentY,lookAtPointX,lookAtPointY,gazeLocationZoneSet
1236,completedLiveItem,testing,NaN,NaN,0.136049,2.737640,NaN,NaN,NaN
1916,completedLiveItem,testing,NaN,NaN,1.164900,1.607064,NaN,NaN,NaN
2607,completedLiveItem,testing,NaN,NaN,0.764043,2.604731,NaN,NaN,NaN
3292,completedLiveItem,testing,NaN,NaN,-0.235415,-0.518413,NaN,NaN,NaN
4326,completedLiveItem,testing,NaN,NaN,0.525879,1.887179,NaN,NaN,NaN
5014,completedLiveItem,testing,NaN,NaN,1.211529,5.552339,NaN,NaN,NaN
5701,completedLiveItem,testing,NaN,NaN,0.850875,0.939704,NaN,NaN,NaN
6389,completedLiveItem,testing,NaN,NaN,-0.113762,2.240209,NaN,NaN,NaN
7421,completedLiveItem,testing,NaN,NaN,1.239873,-0.759900,NaN,NaN,NaN
8115,completedLiveItem,testing,NaN,NaN,-0.059622,2.588714,NaN,NaN,NaN


In [31]:
doc =[]
for i in range(len(begin)):
    sub = df.iloc[begin[i]+1:end[i]]
    doc.append(sub)
    
new = pd.concat(doc)

In [32]:
new.tail()

,eventName,gazeEngineState,gazeLocationName,gazeLocationOnScreen,gazeLocationPercentX,gazeLocationPercentY,lookAtPointX,lookAtPointY,gazeLocationZoneSet
18715,faceVerticesChanged,testing,away,false,0.984635,9.653067,167.774871,148.304015,NaN
18716,faceVerticesChanged,testing,away,false,1.424930,12.483100,238.047093,175.236925,NaN
18717,cameraImageCaptured,testing,NaN,NaN,2.108818,15.723991,NaN,NaN,NaN
18718,faceLeftScene,testing,NaN,NaN,2.108818,15.723991,NaN,NaN,NaN
18719,cameraImageCaptured,testing,NaN,NaN,2.108818,15.723991,NaN,NaN,NaN


In [34]:
new.gazeLocationOnScreen.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

true     91.1%
false     8.9%
Name: gazeLocationOnScreen, dtype: object

In [35]:
new.gazeLocationName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

left     51.7%
right    36.1%
away     12.2%
Name: gazeLocationName, dtype: object